# union of train dataset for summarization

In [1]:
import os
import pandas as pd

In [2]:
ROOT_PATH_SUMMARIZATION = "../data/interim/summarization"
ROOT_DESTINATION_PATH = "../data/processed/summarization"

In [3]:
COSTITUZIONALE_PATH = os.path.join(ROOT_PATH_SUMMARIZATION, "costituzionale")
EUR_LEX_SUM_IT_PATH = os.path.join(ROOT_PATH_SUMMARIZATION, "eur-lex-sum_it")
EUROPARL_PATH = os.path.join(ROOT_PATH_SUMMARIZATION, "euparl")
ITALCASEHOLD = os.path.join(ROOT_PATH_SUMMARIZATION, "itacasehold")

In [4]:
ENCODINGS = ["utf-8", "utf-8-sig", "latin-1", "iso-8859-1"]

In [5]:
import json


def read_json_file(file_path, enc="utf-8"):
    try:
        with open(file_path, "r", encoding=enc) as file:
            data = json.load(file)
            if isinstance(data, list):
                return data
            else:
                # If the JSON file contains a dictionary or other format, put it into a list
                return [data]
    except FileNotFoundError:
        print("File not found. Please provide a valid file path.")
    except json.JSONDecodeError:
        print("Invalid JSON format in the file.")
    except Exception as e:
        print(f"An error occurred: {e}")


# # Example usage:
# file_path = 'example.json'  # Replace this with your JSON file path
# result = read_json_file(file_path)
# if result:
#     print("Contents of JSON file as a list:")
#     print(result)

In [6]:
# os.path.join(COSTITUZIONALE_PATH, "massime_and_pronincie.json")

# ../data/interim/summarization/costituzionale/massime_and_pronincie.json

In [7]:
# read the costituzionale dataset
costituzionale_list_dataset = read_json_file(
    os.path.join(COSTITUZIONALE_PATH, "massime_and_pronincie.json")
)

In [8]:
# read the euro-lex-sum_it dataset

import json


def read_json_line_by_line(file_path):
    data = []
    try:
        with open(file_path, "r") as file:
            for line in file:
                try:
                    # Load each line as a JSON object
                    data.append(json.loads(line))
                    # print(data)  # Or do something else with the data
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON: {e}. Line skipped.")
    except FileNotFoundError:
        print("File not found. Please provide a valid file path.")
    except Exception as e:
        print(f"An error occurred: {e}")

    return data


# for enc in ENCODINGS:
eur_lex_sum_it_list_dataset = read_json_line_by_line(
    os.path.join(EUR_LEX_SUM_IT_PATH, "train.json")
)

In [9]:
# # read the euparl dataset
# euparl_list_dataset = read_json_file(
#     os.path.join(EUROPARL_PATH, "train.json"))

In [10]:
# read italcasehold dataset
italcasehold_list_dataset = read_json_file(
    os.path.join(ITALCASEHOLD, "train_Itacasehold.json")
)

In [11]:
print(type(costituzionale_list_dataset))
print(type(eur_lex_sum_it_list_dataset))
print(type(italcasehold_list_dataset))

<class 'list'>
<class 'list'>
<class 'list'>


In [12]:
print(len(costituzionale_list_dataset))
print(len(eur_lex_sum_it_list_dataset))
print(len(italcasehold_list_dataset))

1
1028
792


In [13]:
type(costituzionale_list_dataset[0])

dict

In [14]:
print(costituzionale_list_dataset[0].keys())

dict_keys(['2021', '2010', '2004', '1975', '1991', '1989', '1987', '1965', '1997', '1967', '1958', '1978', '1981', '2012', '1968', '2009', '1994', '2023', '1996', '1961', '1985', '1962', '1963', '2016', '1972', '1964', '1969', '2022', '1999', '2002', '1956', '2020', '1971', '1995', '1990', '2019', '1976', '1957', '1993', '2007', '1977', '2005', '1979', '2003', '1983', '1966', '1984', '1959', '1960', '2000', '1973', '2006', '2015', '1982', '2011', '2017', '1986', '2018', '2013', '2008', '1970', '1998', '2001', '2014', '1980', '1974', '1992', '1988'])


In [15]:
print(type(costituzionale_list_dataset[0]["2021"]))

<class 'dict'>


In [16]:
print(costituzionale_list_dataset[0]["2021"]["1"].keys())

dict_keys(['testo_pronuncia', 'testo_massima'])


In [17]:
costituzionale_final_list_dataset = []
for anno in costituzionale_list_dataset[0]:
    for num in costituzionale_list_dataset[0][anno]:
        costituzionale_final_list_dataset.append(
            costituzionale_list_dataset[0][anno][num]
        )

In [18]:
print(type(costituzionale_final_list_dataset))

<class 'list'>


In [19]:
len(costituzionale_final_list_dataset)

21790

In [20]:
type(costituzionale_final_list_dataset[0])

dict

In [21]:
print(costituzionale_final_list_dataset[0].keys())

dict_keys(['testo_pronuncia', 'testo_massima'])


In [22]:
print(eur_lex_sum_it_list_dataset[0].keys())
print(italcasehold_list_dataset[0].keys())

dict_keys(['celex_id', 'reference', 'summary'])
dict_keys(['url', 'title', 'doc', 'summary', 'materia'])


In [23]:
merge_data = []

for eur in eur_lex_sum_it_list_dataset:
    merge_data.append({"reference": eur["reference"], "summary": eur["summary"]})

for ita in italcasehold_list_dataset:
    merge_data.append({"reference": ita["doc"], "summary": ita["summary"]})

for cost in costituzionale_final_list_dataset:
    merge_data.append(
        {"reference": cost["testo_pronuncia"], "summary": cost["testo_massima"]}
    )

In [24]:
print(type(merge_data[0]))
print(len(merge_data))

<class 'dict'>
23610


In [25]:
print(merge_data[0].keys())

dict_keys(['reference', 'summary'])


In [26]:
def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created.")
    else:
        print(f"Folder '{folder_path}' already exists.")


def save_list_of_dicts_to_json(data_list, destination_folder, file_name):
    create_folder_if_not_exists(destination_folder)
    destination_path = os.path.join(destination_folder, file_name)
    try:
        with open(destination_path, "w") as file:
            json.dump(data_list, file, indent=4, ensure_ascii=False)
        print(f"Data successfully saved to {destination_path}")
    except Exception as e:
        print(f"Error occurred while saving data: {e}")

In [27]:
save_list_of_dicts_to_json(
    merge_data, ROOT_DESTINATION_PATH, "train_union_summarization.json"
)

Folder '../data/processed/summarization' already exists.
Data successfully saved to ../data/processed/summarization/train_union_summarization.json


In [27]:
ds = pd.read_json(os.path.join(ROOT_DESTINATION_PATH, "train_union_summarization.json"))

In [28]:
print(len(ds))

23610


In [30]:
from datasets import Dataset


def convert_lists_to_strings(x):
    if isinstance(x, list):
        return ",".join(map(str, x))
    return x


df_fixed = ds.applymap(convert_lists_to_strings)

ds_data = Dataset.from_pandas(df_fixed)

/tmp/ipykernel_2276667/3469970038.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_fixed = ds.applymap(convert_lists_to_strings)


In [32]:
df_fixed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23610 entries, 0 to 23609
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   reference  23610 non-null  object
 1   summary    23395 non-null  object
dtypes: object(2)
memory usage: 369.0+ KB


In [33]:
import pandas as pd

# Sample DataFrame (replace this with your DataFrame)
data = {"A": [1, 2, None, 4], "B": ["a", None, "c", "d"], "C": ["x", "y", "z", "w"]}

df = pd.DataFrame(data)

# Drop rows with at least one null value
df_cleaned = df.dropna()

# Display the DataFrame without rows containing null values
print(df_cleaned)

     A  B  C
0  1.0  a  x
3  4.0  d  w


In [31]:
ds_data

Dataset({
    features: ['reference', 'summary'],
    num_rows: 23610
})

# union dataset for semplification

In [99]:
ROOT_PATH_SEMPLIFICATION = "../data/interim/simplification"
ROOT_DESTINATION_PATH_SEMPLIFICATION = "../data/processed/simplification"

ADMIN_IT = "admin_it"
ADMIN_IT2 = "admin_it2"
PACCSS_IT = "paccss_it"
SIMPITIKI = "simpitiki"

In [100]:
import json


def read_json_file(file_path, enc="utf-8"):
    try:
        with open(file_path, "r", encoding=enc) as file:
            data = json.load(file)
            if isinstance(data, list):
                return data
            else:
                # If the JSON file contains a dictionary or other format, put it into a list
                return [data]
    except FileNotFoundError:
        print("File not found. Please provide a valid file path.")
    except json.JSONDecodeError:
        print("Invalid JSON format in the file.")
    except Exception as e:
        print(f"An error occurred: {e}")

In [101]:
admin_it_dataset_op = read_json_file(
    os.path.join(ROOT_PATH_SEMPLIFICATION, ADMIN_IT, "OP.json")
)
admin_it_dataset_rd = read_json_file(
    os.path.join(ROOT_PATH_SEMPLIFICATION, ADMIN_IT, "RD.json")
)
admin_it_dataset_rs = read_json_file(
    os.path.join(ROOT_PATH_SEMPLIFICATION, ADMIN_IT, "RS.json")
)

In [102]:
print(len(admin_it_dataset_op))

1


In [103]:
admin_it2_dataset = read_json_file(
    os.path.join(ROOT_PATH_SEMPLIFICATION, ADMIN_IT2, "admin-it-l2.json")
)
paccss_it_dataset = read_json_file(
    os.path.join(ROOT_PATH_SEMPLIFICATION, PACCSS_IT, "paccss_it.json")
)
simpitiki_dataset = read_json_file(
    os.path.join(ROOT_PATH_SEMPLIFICATION, SIMPITIKI, "simpitiki-v2.json")
)

In [104]:
print(admin_it2_dataset[0].keys())
print(paccss_it_dataset[0].keys())
print(simpitiki_dataset[0].keys())

dict_keys(['original', 'simplification', 'l2_semp'])
dict_keys(['Sentence_1', 'Sentence_2', 'Cosine_Similarity', 'Confidence', 'Readability_1', 'Readability_2', '(Readability_1-Readability_2)'])
dict_keys(['type', 'origin', 'before', 'after'])


In [105]:
final_semplification_dataset = []

for a2 in admin_it2_dataset:
    final_semplification_dataset.append(
        {"original": a2["original"], "simplified": a2["simplification"]}
    )
    final_semplification_dataset.append(
        {"original": a2["original"], "simplified": a2["l2_semp"]}
    )
    final_semplification_dataset.append(
        {"original": a2["simplification"], "simplified": a2["l2_semp"]}
    )

for pacs in paccss_it_dataset:
    final_semplification_dataset.append(
        {"original": pacs["Sentence_1"], "simplified": pacs["Sentence_2"]}
    )

for swi in simpitiki_dataset:
    final_semplification_dataset.append(
        {"original": swi["before"], "simplified": swi["after"]}
    )

In [106]:
for op in admin_it_dataset_op[0]:
    final_semplification_dataset.append(
        {
            "original": admin_it_dataset_op[0][op][0],
            "simplified": admin_it_dataset_op[0][op][1],
        }
    )

    for rs in admin_it_dataset_rs[0]:
        final_semplification_dataset.append(
            {
                "original": admin_it_dataset_rs[0][rs][0],
                "simplified": admin_it_dataset_rs[0][rs][1],
            }
        )

for rd in admin_it_dataset_rd[0]:
    final_semplification_dataset.append(
        {
            "original": admin_it_dataset_rd[0][rd][0],
            "simplified": admin_it_dataset_rd[0][rd][1],
        }
    )

In [107]:
save_list_of_dicts_to_json(
    merge_data, ROOT_DESTINATION_PATH_SEMPLIFICATION, "train_union_simplification.json"
)

Folder '../data/processed/simplification' created.
Data successfully saved to ../data/processed/simplification/train_union_simplification.json
